# 4. Predict-Image-Similar-FCNN-Binary
For landmark-recognition-2019 algorithm validation

## Run name

In [1]:
import time

project_name = 'Dog-Breed'
model_name = 'Dog-Breed_3-Image-Similar-FCNN-Binary_20190501-194400'
step_name = '4-Predict-%s' % model_name
time_str = time.strftime("%Y%m%d-%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)
t0 = time.time()

run_name: Dog-Breed_4-Predict-Dog-Breed_3-Image-Similar-FCNN-Binary_20190501-194400_20190502-092437


## Important params

In [2]:
import multiprocessing

cpu_amount = multiprocessing.cpu_count()
print('cpu_amount: ', cpu_amount)

cpu_amount:  4


## Import PKGs

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from IPython.display import display

import os
import sys
import gc
import math
import shutil
import zipfile
import pickle
import h5py

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [4]:
import keras
from keras.utils import Sequence
from keras.layers import *
from keras.models import *
from keras.applications import *
from keras.optimizers import *
from keras.regularizers import *
from keras.preprocessing.image import *
from keras.applications.inception_v3 import preprocess_input

Using TensorFlow backend.


## Project folders

In [5]:
cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')

org_train_folder = os.path.join(input_folder, 'org_train')
org_test_folder = os.path.join(input_folder, 'org_test')
train_folder = os.path.join(input_folder, 'data_train')
val_folder = os.path.join(input_folder, 'data_val')
test_folder = os.path.join(input_folder, 'data_test')
test_sub_folder = os.path.join(test_folder, 'test')

vgg16_feature_file = os.path.join(feature_folder, 'feature_wrapper_171023.h5')
train_csv_file = os.path.join(input_folder, 'train.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
sample_submission_folder = os.path.join(input_folder, 'sample_submission.csv')
print(vgg16_feature_file)
print(train_csv_file)
print(test_csv_file)
print(sample_submission_folder)

D:\Kaggle\dog-breed-identification\feature\feature_wrapper_171023.h5
D:\Kaggle\dog-breed-identification\input\train.csv
D:\Kaggle\dog-breed-identification\input\test.csv
D:\Kaggle\dog-breed-identification\input\sample_submission.csv


## Load feature

In [6]:
with h5py.File(vgg16_feature_file, 'r') as h:
    x_train = np.array(h['train'])
    y_train = np.array(h['train_label'])
    x_val = np.array(h['val'])
    y_val = np.array(h['val_label'])


In [7]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(9199, 512)
(9199,)
(1023, 512)
(1023,)


## Load model

In [8]:
model_file = os.path.join(model_folder, '%s.h5' % model_name)
print(model_file)
model = load_model(model_file)

D:\Kaggle\dog-breed-identification\model\Dog-Breed_3-Image-Similar-FCNN-Binary_20190501-194400.h5
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [9]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 512)          0                                            
__________________________________________________________________________________________________
library_input (InputLayer)      (None, 512)          0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 1024)         0           main_input[0][0]                 
                                                                 library_input[0][0]              
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1024)         1049600     concatenate_1[0][0]              
__________

## Predict

### single item

In [10]:
idx = 0
item_main_x = np.array([x_train[idx]]*x_train.shape[0])
item_x_train = {
    'main_input': item_main_x,
    'library_input': x_train
}
y_proba = model.predict(item_x_train, batch_size=1024)
print(y_train[idx], np.argmax(y_proba), '-->', y_train[np.argmax(y_proba)], [y_train[item[0]] for item in np.argsort(y_proba, axis=0)[-10:]])

0 2372 --> 30 [0, 0, 54, 20, 0, 62, 0, 54, 85, 30]


### topn

In [11]:
topn = 10
amount = 10
top1_count = 0
topn_count = 0
for idx in range(amount):
    item_main_x = np.array([x_train[idx]]*x_train.shape[0])
    item_x_train = {
        'main_input': item_main_x,
        'library_input': x_train
    }
    y_proba = model.predict(item_x_train, batch_size=1024)
    item_y_train = y_train[idx]
    top1_pred = y_train[np.argmax(y_proba)]
    topn_pred_arr = [y_train[item[0]] for item in np.argsort(y_proba, axis=0)[-topn:]]
    if item_y_train == top1_pred:
        top1_count += 1
    if item_y_train in topn_pred_arr:
        topn_count += 1
    print(item_y_train, '-->', top1_pred, topn_pred_arr)

print('*' * 80)
print(top1_count, '%.2f' % (top1_count/amount))
print(topn_count, '%.2f' % (topn_count/amount))

0 --> 30 [0, 0, 54, 20, 0, 62, 0, 54, 85, 30]
0 --> 0 [96, 30, 108, 109, 1, 0, 23, 0, 0, 0]
0 --> 1 [97, 1, 1, 1, 106, 30, 1, 78, 1, 1]
0 --> 6 [26, 0, 0, 0, 6, 6, 79, 119, 119, 6]
0 --> 0 [85, 0, 0, 0, 96, 26, 0, 26, 96, 0]
0 --> 0 [0, 0, 96, 0, 0, 26, 26, 0, 0, 0]
0 --> 0 [64, 0, 0, 76, 0, 78, 109, 20, 3, 0]
0 --> 59 [107, 68, 77, 68, 0, 48, 3, 3, 110, 59]
0 --> 109 [23, 0, 6, 119, 96, 111, 26, 0, 26, 109]
0 --> 0 [0, 20, 108, 85, 0, 0, 0, 0, 0, 0]
********************************************************************************
5 0.50
9 0.90


### weighted topn

In [12]:
def get_weight_topn(y_proba, topn, y_data):
    y_proba_argsorted = [(y_data[item[0]], y_proba[item[0]][0]) for item in np.argsort(y_proba, axis=0)[-topn:]]
    class_score_dict = {}
    for item_class, item_proba in y_proba_argsorted:
        if item_class in class_score_dict:
            class_score_dict[item_class] += item_proba
        else:
            class_score_dict[item_class] = item_proba
    class_score_arr = list(class_score_dict.items())
    class_score_arr = sorted(class_score_arr, key=lambda x: x[1])
    topn_pred_arr = [item[0] for item in class_score_arr]
    class_score_arr = [round(item[1], 2) for item in class_score_arr]
    return topn_pred_arr, class_score_arr

y_proba = np.array([[0.1], [0.2], [0.3], [0.2], [0.2], [0.2]])
topn = 6
y_data = np.array([0, 0, 1, 1, 2, 2])
print(y_proba, type(y_proba))
print(topn)
print(y_data, type(y_data))
get_weight_topn(y_proba, topn, y_data)

[[0.1]
 [0.2]
 [0.3]
 [0.2]
 [0.2]
 [0.2]] <class 'numpy.ndarray'>
6
[0 0 1 1 2 2] <class 'numpy.ndarray'>


([0, 2, 1], [0.3, 0.4, 0.5])

In [13]:
topn = 10
amount = 10
top1_count = 0
topn_1_count = 0
topn_count = 0
for idx in range(amount):
    item_main_x = np.array([x_train[idx]]*x_train.shape[0])
    item_x_train = {
        'main_input': item_main_x,
        'library_input': x_train
    }
    y_proba = model.predict(item_x_train, batch_size=1024)
    item_y_train = y_train[idx]
    top1_pred = y_train[np.argmax(y_proba)]
#     topn_pred_arr = [y_train[item[0]] for item in np.argsort(y_proba, axis=0)[-topn:]]
    topn_pred_arr, class_score_arr = get_weight_topn(y_proba, topn, y_train)
    topn_1_pred = topn_pred_arr[-1]
    if item_y_train == top1_pred:
        top1_count += 1
    if item_y_train == topn_1_pred:
        topn_1_count += 1
    if item_y_train in topn_pred_arr:
        topn_count += 1
    print(item_y_train, '-->', top1_pred, '\t', topn_1_pred, topn_pred_arr, class_score_arr)

print('*' * 80)
print(top1_count, '%.2f' % (top1_count/amount))
print(topn_1_count, '%.2f' % (topn_1_count/amount))
print(topn_count, '%.2f' % (topn_count/amount))

0 --> 30 	 0 [20, 62, 85, 30, 54, 0] [0.98, 0.98, 0.99, 0.99, 1.96, 3.92]
0 --> 0 	 0 [96, 30, 108, 109, 1, 23, 0] [0.95, 0.95, 0.95, 0.95, 0.96, 0.96, 3.86]
0 --> 1 	 1 [97, 106, 30, 78, 1] [0.96, 0.97, 0.97, 0.97, 5.8]
0 --> 6 	 6 [26, 79, 119, 0, 6] [0.98, 0.98, 1.97, 2.94, 2.96]
0 --> 0 	 0 [85, 96, 26, 0] [0.99, 1.98, 1.98, 4.94]
0 --> 0 	 0 [96, 26, 0] [0.97, 1.95, 6.83]
0 --> 0 	 0 [64, 76, 78, 109, 20, 3, 0] [0.96, 0.96, 0.97, 0.97, 0.97, 0.97, 3.87]
0 --> 59 	 3 [107, 77, 0, 48, 110, 59, 68, 3] [0.88, 0.89, 0.9, 0.91, 0.93, 0.95, 1.78, 1.83]
0 --> 109 	 26 [23, 6, 119, 96, 111, 109, 0, 26] [0.96, 0.96, 0.96, 0.97, 0.97, 0.97, 1.93, 1.93]
0 --> 0 	 0 [20, 108, 85, 0] [0.98, 0.98, 0.98, 6.91]
********************************************************************************
5 0.50
6 0.60
9 0.90


### encupsolution

In [14]:
def get_single_pred(idx, main_x, libary_x, topn, batch_size=1024):
    item_main_x = np.array([main_x[idx]]*libary_x.shape[0])
    item_x = {
        'main_input': item_main_x,
        'library_input': libary_x
    }
    y_proba = model.predict(item_x, batch_size=batch_size)
    top1_pred = y_train[np.argmax(y_proba)]
    topn_pred_arr = [y_train[item[0]] for item in np.argsort(y_proba, axis=0)[-topn:]]
    weighted_topn_pred_arr, class_score_arr = get_weight_topn(y_proba, topn, y_train)
    weighted_top1_pred = weighted_topn_pred_arr[-1]
    return top1_pred, topn_pred_arr, weighted_top1_pred, weighted_topn_pred_arr, class_score_arr

idx = 0
topn = 10
# train
print(y_train[idx], get_single_pred(idx, x_train, x_train, topn, batch_size=1024))
# val
print(y_val[idx], get_single_pred(idx, x_val, x_train, topn, batch_size=1024))

0 (30, [0, 0, 54, 20, 0, 62, 0, 54, 85, 30], 0, [20, 62, 85, 30, 54, 0], [0.98, 0.98, 0.99, 0.99, 1.96, 3.92])
0 (54, [54, 54, 62, 62, 87, 62, 54, 30, 54, 54], 54, [87, 30, 62, 54], [0.99, 0.99, 2.96, 4.95])


In [15]:
topn = 10
amount = 10
top1_count = 0
topn_1_count = 0
topn_count = 0
for idx in range(amount):
    top1_pred, topn_pred_arr, weighted_top1_pred, weighted_topn_pred_arr, class_score_arr = get_single_pred(idx, x_train, x_train, topn, batch_size=1024)
    
    item_y_data = y_train[idx]
    if item_y_data == top1_pred:
        top1_count += 1
    if item_y_data == weighted_top1_pred:
        topn_1_count += 1
    if item_y_data in topn_pred_arr:
        topn_count += 1
    print(item_y_data, '-->', top1_pred, weighted_top1_pred, '\t', weighted_topn_pred_arr, class_score_arr)

print('*' * 80)
print(top1_count, '%.2f' % (top1_count/amount))
print(topn_1_count, '%.2f' % (topn_1_count/amount))
print(topn_count, '%.2f' % (topn_count/amount))

0 --> 30 0 	 [20, 62, 85, 30, 54, 0] [0.98, 0.98, 0.99, 0.99, 1.96, 3.92]
0 --> 0 0 	 [96, 30, 108, 109, 1, 23, 0] [0.95, 0.95, 0.95, 0.95, 0.96, 0.96, 3.86]
0 --> 1 1 	 [97, 106, 30, 78, 1] [0.96, 0.97, 0.97, 0.97, 5.8]
0 --> 6 6 	 [26, 79, 119, 0, 6] [0.98, 0.98, 1.97, 2.94, 2.96]
0 --> 0 0 	 [85, 96, 26, 0] [0.99, 1.98, 1.98, 4.94]
0 --> 0 0 	 [96, 26, 0] [0.97, 1.95, 6.83]
0 --> 0 0 	 [64, 76, 78, 109, 20, 3, 0] [0.96, 0.96, 0.97, 0.97, 0.97, 0.97, 3.87]
0 --> 59 3 	 [107, 77, 0, 48, 110, 59, 68, 3] [0.88, 0.89, 0.9, 0.91, 0.93, 0.95, 1.78, 1.83]
0 --> 109 26 	 [23, 6, 119, 96, 111, 109, 0, 26] [0.96, 0.96, 0.96, 0.97, 0.97, 0.97, 1.93, 1.93]
0 --> 0 0 	 [20, 108, 85, 0] [0.98, 0.98, 0.98, 6.91]
********************************************************************************
5 0.50
6 0.60
9 0.90


In [16]:
topn = 10
amount = 10
top1_count = 0
topn_1_count = 0
topn_count = 0
for idx in range(amount):
    top1_pred, topn_pred_arr, weighted_top1_pred, weighted_topn_pred_arr, class_score_arr = get_single_pred(idx, x_val, x_train, topn, batch_size=1024)
    
    item_y_data = y_val[idx]
    if item_y_data == top1_pred:
        top1_count += 1
    if item_y_data == weighted_top1_pred:
        topn_1_count += 1
    if item_y_data in topn_pred_arr:
        topn_count += 1
    print(item_y_data, '-->', top1_pred, weighted_top1_pred, '\t', weighted_topn_pred_arr, class_score_arr)

print('*' * 80)
print(top1_count, '%.2f' % (top1_count/amount))
print(topn_1_count, '%.2f' % (topn_1_count/amount))
print(topn_count, '%.2f' % (topn_count/amount))

0 --> 54 54 	 [87, 30, 62, 54] [0.99, 0.99, 2.96, 4.95]
0 --> 0 0 	 [26, 119, 96, 0] [0.99, 0.99, 1.98, 5.93]
0 --> 70 0 	 [85, 20, 70, 109, 0] [0.97, 0.98, 1.96, 2.91, 2.92]
0 --> 54 58 	 [78, 105, 54, 20, 58] [0.99, 0.99, 1.0, 2.97, 3.96]
0 --> 70 87 	 [73, 85, 65, 70, 0, 30, 87] [0.98, 0.98, 0.99, 0.99, 1.97, 1.97, 1.97]
0 --> 0 0 	 [85, 0] [0.97, 8.74]
0 --> 0 0 	 [96, 26, 109, 0] [0.98, 0.98, 0.98, 6.87]
0 --> 0 0 	 [26, 0] [0.98, 8.83]
0 --> 85 73 	 [83, 110, 85, 70, 73] [0.97, 0.98, 0.98, 1.96, 4.89]
0 --> 48 48 	 [107, 3, 102, 20, 0, 48] [0.93, 0.93, 0.94, 0.94, 2.81, 2.84]
********************************************************************************
4 0.40
5 0.50
7 0.70


In [17]:
acc10000 = int(topn_count/amount*10000)
# acc10000 = 10000 # for test
# print(acc10000)
if acc10000 == 10000:
    run_name_acc = '%s_%05d' % (run_name, acc10000)
else:
    run_name_acc = '%s_%04d' % (run_name, acc10000)
print(run_name_acc)

Dog-Breed_4-Predict-Dog-Breed_3-Image-Similar-FCNN-Binary_20190501-194400_20190502-092437_7000


In [18]:
print('Time elapsed: %.1fs' % (time.time() - t0))
print(run_name_acc)

Time elapsed: 62.4s
Dog-Breed_4-Predict-Dog-Breed_3-Image-Similar-FCNN-Binary_20190501-194400_20190502-092437_7000
